<a href="https://colab.research.google.com/github/Ujwalgowdru/InfosysSpacetraffic/blob/main/training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

# Function to load and preprocess the data
def load_and_preprocess_data(file_path):
    # Load the CSV file
    data = pd.read_csv(file_path)

    # Convert 'Timestamp' to datetime and extract useful features
    data['Timestamp'] = pd.to_datetime(data['Timestamp'])
    data['Hour'] = data['Timestamp'].dt.hour
    data['Day'] = data['Timestamp'].dt.day
    data['Month'] = data['Timestamp'].dt.month

    # Encode categorical columns
    label_encoders = {}
    categorical_columns = ['Location', 'Object_Type', 'Peak_Time']
    for col in categorical_columns:
        le = LabelEncoder()
        data[col] = le.fit_transform(data[col])
        label_encoders[col] = le

    # Normalize numerical columns
    scaler = MinMaxScaler()
    data[['Traffic_Density', 'Hour', 'Day', 'Month']] = scaler.fit_transform(
        data[['Traffic_Density', 'Hour', 'Day', 'Month']]
    )

    # Prepare features (X) and target (y)
    features = ['Location', 'Object_Type', 'Peak_Time', 'Hour', 'Day', 'Month']
    X = data[features].values
    y = data['Traffic_Density'].values

    # Reshape X for RNN input (samples, timesteps, features)
    X = X.reshape((X.shape[0], 1, X.shape[1]))

    return X, y

# Load the data
file_path = '/content/drive/MyDrive/Resume/space_traffic.csv'  # Replace with your file path
X, y = load_and_preprocess_data(file_path)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Build the RNN model
def build_rnn_model(input_shape):
    model = Sequential([
        LSTM(64, input_shape=input_shape, return_sequences=False),
        Dropout(0.2),
        Dense(32, activation='relu'),
        Dropout(0.2),
        Dense(1, activation='linear')  # Output layer for regression
    ])
    model.compile(optimizer='adam', loss='mse', metrics=['mae'])
    return model

# Initialize the model
model = build_rnn_model((X_train.shape[1], X_train.shape[2]))

# Train the model
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2, verbose=1)

# Evaluate the model
loss, mae = model.evaluate(X_test, y_test, verbose=1)
print(f"Test Loss: {loss}, Test MAE: {mae}")

# Save the model
model.save('space_traffic_rnn_model.h5')


Epoch 1/50


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


20/20 ━━━━━━━━━━━━━━━━━━━━ 7s 80ms/step - loss: 0.2317 - mae: 0.3890 - val_loss: 0.0863 - val_mae: 0.2432
Epoch 2/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 0.1107 - mae: 0.2706 - val_loss: 0.0785 - val_mae: 0.2379
Epoch 3/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 0.1013 - mae: 0.2610 - val_loss: 0.0812 - val_mae: 0.2405
Epoch 4/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 0.1029 - mae: 0.2710 - val_loss: 0.0748 - val_mae: 0.2325
Epoch 5/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 0.0976 - mae: 0.2632 - val_loss: 0.0763 - val_mae: 0.2339
Epoch 6/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - loss: 0.0920 - mae: 0.2523 - val_loss: 0.0746 - val_mae: 0.2314
Epoch 7/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0946 - mae: 0.2589 - val_loss: 0.0764 - val_mae: 0.2334
Epoch 8/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0981 - mae: 0.2618 - val_loss: 0.0777 - val_mae: 0.2342
Epoch 9/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0978 - mae: 0.263

Test Loss: 0.08192344009876251, Test MAE: 0.24838218092918396
